In [3]:
from clickhouse_connect import get_client
import pandas as pd
client = get_client(
    host="chdb.ruogu.work",
    port=8123,
    username="user40",
    password="fb4a4d977b118e1540d6577fe41958f3"
)

### 逐笔数据复现盘口数据

In [ ]:
# 输入code，date以及需要复现的时间
time = '11:30:00'
code = '000006'
date = '20250930'
# Sql重建盘口
sql = f'''
WITH 
    # 1. 设定目标时间
    target_t AS (SELECT Tit('{time}') as t),

    # 2. 提取所有"限价委托单" (作为盘口的底账)
    # 使用 seqno 作为唯一标识
    limit_orders AS (
        SELECT 
            toInt64(seqno) as seqno,     # 统一转为Int64
            price, 
            toString(side) as side, 
            toInt64(volume) as orig_vol
        FROM stock_base.zb
        WHERE date = '{date}' 
          AND code = '{code}'
          AND time_int <= (SELECT t FROM target_t)
          AND toString(type) = '2'       # 必须是限价单
    ),

    # 3. 计算所有"消耗量" (成交 + 撤单)
    consumptions AS (
        # 3.1 撤单消耗 (用 cbidno 关联买方撤单)
        SELECT 
            toInt64(cbidno) as orig_seqno, 
            SUM(toInt64(volume)) as consumed_vol
        FROM stock_base.zb
        WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
          AND toString(trade_flag) = '4'
          AND cbidno > 0
        GROUP BY cbidno
        
        UNION ALL
        
        # 3.2 撤单消耗 (用 caskno 关联卖方撤单)
        SELECT 
            toInt64(caskno) as orig_seqno, 
            SUM(toInt64(volume)) as consumed_vol
        FROM stock_base.zb
        WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
          AND toString(trade_flag) = '4'
          AND caskno > 0
        GROUP BY caskno
        
        UNION ALL
        
        # 3.3 成交消耗 - 买方 (用 cbidno 关联)
        SELECT 
            toInt64(cbidno) as orig_seqno, 
            SUM(toInt64(volume)) as consumed_vol
        FROM stock_base.zb
        WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
          AND toString(trade_flag) = 'F'
          AND cbidno > 0
        GROUP BY cbidno
        
        UNION ALL
        
        # 3.4 成交消耗 - 卖方 (用 caskno 关联)
        SELECT 
            toInt64(caskno) as orig_seqno, 
            SUM(toInt64(volume)) as consumed_vol
        FROM stock_base.zb
        WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
          AND toString(trade_flag) = 'F'
          AND caskno > 0
        GROUP BY caskno
    ),

    # 4. 汇总每个订单的消耗量
    total_consumed AS (
        SELECT orig_seqno, SUM(consumed_vol) as total_used
        FROM consumptions
        GROUP BY orig_seqno
    ),

    # 5. 计算剩余挂单量 (Resting Volume)
    order_book_snapshot AS (
        SELECT 
            L.price,
            L.side,
            (L.orig_vol - COALESCE(C.total_used, 0)) as resting_vol
        FROM limit_orders L
        LEFT JOIN total_consumed C ON L.seqno = C.orig_seqno
        WHERE (L.orig_vol - COALESCE(C.total_used, 0)) > 0
    ),

    # 6. 聚合生成盘口
    levels_agg AS (
        SELECT 
            price,
            side,
            SUM(resting_vol) as level_vol
        FROM order_book_snapshot
        GROUP BY price, side
    ),
    
    # 7. 买盘排序
    bids_ranked AS (
        SELECT 
            'bid' as type,
            row_number() OVER (ORDER BY price DESC) as level, 
            price, 
            level_vol 
        FROM levels_agg 
        WHERE side = 'B'
    ),
    
    # 8. 卖盘排序
    asks_ranked AS (
        SELECT 
            'ask' as type,
            row_number() OVER (ORDER BY price ASC) as level, 
            price, 
            level_vol 
        FROM levels_agg 
        WHERE side = 'S'
    )

SELECT type, level, price, level_vol as volume FROM bids_ranked WHERE level <= 10
UNION ALL
SELECT type, level, price, level_vol as volume FROM asks_ranked WHERE level <= 10
ORDER BY type DESC, level ASC
'''
df_reconstructed = client.query_df(sql)
print("SQL重建的盘口数据:")
df_reconstructed

SQL重建的盘口数据:


,type,level,price,volume
0,bid,1,10.29,21028400
1,bid,2,10.28,60600
2,bid,3,10.27,118700
3,bid,4,10.26,15500
4,bid,5,10.25,53000
5,bid,6,10.24,1200
6,bid,7,10.22,6500
7,bid,8,10.21,22000
8,bid,9,10.20,80900
9,bid,10,10.19,34500


### 获取真实tk快照数据进行对比

In [6]:
sql_tk = f'''
    SELECT 
        bid1, bidv1, ask1, askv1,
        bid2, bidv2, ask2, askv2,
        bid3, bidv3, ask3, askv3,
        bid4, bidv4, ask4, askv4,
        bid5, bidv5, ask5, askv5,
        bid6, bidv6, ask6, askv6,
        bid7, bidv7, ask7, askv7,
        bid8, bidv8, ask8, askv8,
        bid9, bidv9, ask9, askv9,
        bid10, bidv10, ask10, askv10
    FROM stock_base.tk 
    WHERE date = '{date}' 
        AND code = '{code}'
        AND time_int = Tit('{time}')
    LIMIT 1
'''
df_tk_real = client.query_df(sql_tk)
print("真实tk盘口数据:")
df_tk_real.T

真实tk盘口数据:


,0
bid1,6.88
bidv1,383296
ask1,6.89
askv1,394500
bid2,6.87
bidv2,485700
ask2,6.9
askv2,2081300
bid3,6.86
bidv3,236800


### 对比复现数据与真实数据

In [ ]:
def compare_orderbook(df_reconstructed, df_tk_real):
    # 对比复现的盘口与真实tk数据
    if len(df_tk_real) == 0 or len(df_reconstructed) == 0:
        print("数据为空，无法对比")
        return None
    
    tk = df_tk_real.iloc[0]
    results = []
    
    # 从df_reconstructed提取买卖盘
    bids = df_reconstructed[df_reconstructed['type'] == 'bid'].sort_values('level')
    asks = df_reconstructed[df_reconstructed['type'] == 'ask'].sort_values('level')
    
    for i in range(1, 11):
        # 买盘对比 (注意tk表的列名是bidv而不是bidvol)
        real_bid_price = tk.get(f'bid{i}', 0)
        real_bid_vol = tk.get(f'bidv{i}', 0)
        
        calc_bid = bids[bids['level'] == i]
        calc_bid_price = calc_bid['price'].values[0] if len(calc_bid) > 0 else 0
        calc_bid_vol = calc_bid['volume'].values[0] if len(calc_bid) > 0 else 0
        
        price_match = abs(real_bid_price - calc_bid_price) < 0.001
        vol_match = real_bid_vol == calc_bid_vol
        
        results.append({
            '字段': f'买{i}价', '真实值': real_bid_price, '计算值': calc_bid_price, 
            '匹配': '✓' if price_match else '✗'
        })
        results.append({
            '字段': f'买{i}量', '真实值': real_bid_vol, '计算值': calc_bid_vol,
            '匹配': '✓' if vol_match else '✗'
        })
        
        # 卖盘对比 (注意tk表的列名是askv而不是askvol)
        real_ask_price = tk.get(f'ask{i}', 0)
        real_ask_vol = tk.get(f'askv{i}', 0)
        
        calc_ask = asks[asks['level'] == i]
        calc_ask_price = calc_ask['price'].values[0] if len(calc_ask) > 0 else 0
        calc_ask_vol = calc_ask['volume'].values[0] if len(calc_ask) > 0 else 0
        
        price_match = abs(real_ask_price - calc_ask_price) < 0.001
        vol_match = real_ask_vol == calc_ask_vol
        
        results.append({
            '字段': f'卖{i}价', '真实值': real_ask_price, '计算值': calc_ask_price,
            '匹配': '✓' if price_match else '✗'
        })
        results.append({
            '字段': f'卖{i}量', '真实值': real_ask_vol, '计算值': calc_ask_vol,
            '匹配': '✓' if vol_match else '✗'
        })
    
    df_cmp = pd.DataFrame(results)
    correct = (df_cmp['匹配'] == '✓').sum()
    total = len(df_cmp)
    print(f"\n正确率: {correct}/{total} = {correct/total*100:.2f}%")
    
    return df_cmp

df_compare = compare_orderbook(df_reconstructed, df_tk_real)
df_compare

### 批量测试：随机抽取10只深交所股票计算复现正确率

In [11]:
import random

# 固定参数
time = '09:33:00'
date = '20250930'

# 1. 从数据库获取深交所股票代码列表（深交所股票代码以0、2、3开头）
sql_codes = f'''
    SELECT DISTINCT code 
    FROM stock_base.tk 
    WHERE date = '{date}'
      AND (code LIKE '0%' OR code LIKE '2%' OR code LIKE '3%')
    ORDER BY code
'''
df_codes = client.query_df(sql_codes)
all_codes = df_codes['code'].tolist()
print(f"深交所股票总数: {len(all_codes)}")

# 2. 随机抽取10只股票
sample_codes = random.sample(all_codes, min(10, len(all_codes)))
print(f"随机抽取的股票: {sample_codes}")

# 3. 定义批量计算函数
def batch_compare(codes, time, date, client):
    """批量计算多只股票的复现正确率"""
    all_results = []
    
    for code in codes:
        # 重建盘口
        sql = f'''
        WITH 
            target_t AS (SELECT Tit('{time}') as t),
            limit_orders AS (
                SELECT 
                    toInt64(seqno) as seqno,
                    price, 
                    toString(side) as side, 
                    toInt64(volume) as orig_vol
                FROM stock_base.zb
                WHERE date = '{date}' 
                  AND code = '{code}'
                  AND time_int <= (SELECT t FROM target_t)
                  AND toString(type) = '2'
            ),
            consumptions AS (
                SELECT toInt64(cbidno) as orig_seqno, SUM(toInt64(volume)) as consumed_vol
                FROM stock_base.zb
                WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
                  AND toString(trade_flag) = '4' AND cbidno > 0
                GROUP BY cbidno
                UNION ALL
                SELECT toInt64(caskno) as orig_seqno, SUM(toInt64(volume)) as consumed_vol
                FROM stock_base.zb
                WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
                  AND toString(trade_flag) = '4' AND caskno > 0
                GROUP BY caskno
                UNION ALL
                SELECT toInt64(cbidno) as orig_seqno, SUM(toInt64(volume)) as consumed_vol
                FROM stock_base.zb
                WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
                  AND toString(trade_flag) = 'F' AND cbidno > 0
                GROUP BY cbidno
                UNION ALL
                SELECT toInt64(caskno) as orig_seqno, SUM(toInt64(volume)) as consumed_vol
                FROM stock_base.zb
                WHERE date = '{date}' AND code = '{code}' AND time_int <= (SELECT t FROM target_t)
                  AND toString(trade_flag) = 'F' AND caskno > 0
                GROUP BY caskno
            ),
            total_consumed AS (
                SELECT orig_seqno, SUM(consumed_vol) as total_used
                FROM consumptions
                GROUP BY orig_seqno
            ),
            order_book_snapshot AS (
                SELECT L.price, L.side, (L.orig_vol - COALESCE(C.total_used, 0)) as resting_vol
                FROM limit_orders L
                LEFT JOIN total_consumed C ON L.seqno = C.orig_seqno
                WHERE (L.orig_vol - COALESCE(C.total_used, 0)) > 0
            ),
            levels_agg AS (
                SELECT price, side, SUM(resting_vol) as level_vol
                FROM order_book_snapshot
                GROUP BY price, side
            ),
            bids_ranked AS (
                SELECT 'bid' as type, row_number() OVER (ORDER BY price DESC) as level, price, level_vol 
                FROM levels_agg WHERE side = 'B'
            ),
            asks_ranked AS (
                SELECT 'ask' as type, row_number() OVER (ORDER BY price ASC) as level, price, level_vol 
                FROM levels_agg WHERE side = 'S'
            )
        SELECT type, level, price, level_vol as volume FROM bids_ranked WHERE level <= 10
        UNION ALL
        SELECT type, level, price, level_vol as volume FROM asks_ranked WHERE level <= 10
        ORDER BY type DESC, level ASC
        '''
        
        try:
            df_reconstructed = client.query_df(sql)
            
            # 获取真实数据
            sql_tk = f'''
                SELECT bid1, bidv1, ask1, askv1, bid2, bidv2, ask2, askv2,
                       bid3, bidv3, ask3, askv3, bid4, bidv4, ask4, askv4,
                       bid5, bidv5, ask5, askv5, bid6, bidv6, ask6, askv6,
                       bid7, bidv7, ask7, askv7, bid8, bidv8, ask8, askv8,
                       bid9, bidv9, ask9, askv9, bid10, bidv10, ask10, askv10
                FROM stock_base.tk 
                WHERE date = '{date}' AND code = '{code}' AND time_int = Tit('{time}')
                LIMIT 1
            '''
            df_tk_real = client.query_df(sql_tk)
            
            if len(df_tk_real) == 0 or len(df_reconstructed) == 0:
                all_results.append({'股票代码': code, '正确字段数': 0, '总字段数': 40, '正确率': 0.0})
                continue
            
            # 计算正确率
            tk = df_tk_real.iloc[0]
            bids = df_reconstructed[df_reconstructed['type'] == 'bid'].sort_values('level')
            asks = df_reconstructed[df_reconstructed['type'] == 'ask'].sort_values('level')
            
            correct = 0
            for i in range(1, 11):
                # 买盘
                real_bid_price = tk.get(f'bid{i}', 0)
                real_bid_vol = tk.get(f'bidv{i}', 0)
                calc_bid = bids[bids['level'] == i]
                calc_bid_price = calc_bid['price'].values[0] if len(calc_bid) > 0 else 0
                calc_bid_vol = calc_bid['volume'].values[0] if len(calc_bid) > 0 else 0
                if abs(real_bid_price - calc_bid_price) < 0.001: correct += 1
                if real_bid_vol == calc_bid_vol: correct += 1
                
                # 卖盘
                real_ask_price = tk.get(f'ask{i}', 0)
                real_ask_vol = tk.get(f'askv{i}', 0)
                calc_ask = asks[asks['level'] == i]
                calc_ask_price = calc_ask['price'].values[0] if len(calc_ask) > 0 else 0
                calc_ask_vol = calc_ask['volume'].values[0] if len(calc_ask) > 0 else 0
                if abs(real_ask_price - calc_ask_price) < 0.001: correct += 1
                if real_ask_vol == calc_ask_vol: correct += 1
            
            all_results.append({
                '股票代码': code, 
                '正确字段数': correct, 
                '总字段数': 40, 
                '正确率': correct / 40 * 100
            })
            print(f"{code}: {correct}/40 = {correct/40*100:.2f}%")
            
        except Exception as e:
            print(f"{code}: 出错 - {e}")
            all_results.append({'股票代码': code, '正确字段数': 0, '总字段数': 40, '正确率': 0.0})
    
    return pd.DataFrame(all_results)

# 4. 执行批量测试
df_batch_results = batch_compare(sample_codes, time, date, client)
print(f"\n========== 汇总结果 ==========")
print(f"平均正确率: {df_batch_results['正确率'].mean():.2f}%")
df_batch_results

深交所股票总数: 2875
随机抽取的股票: ['300862', '002967', '300789', '300368', '300570', '002350', '300294', '300591', '000158', '002949']
002967: 38/40 = 95.00%
300789: 40/40 = 100.00%
300368: 0/40 = 0.00%
300570: 5/40 = 12.50%
002350: 40/40 = 100.00%
300294: 20/40 = 50.00%
300591: 37/40 = 92.50%
000158: 36/40 = 90.00%
002949: 39/40 = 97.50%

========== 汇总结果 ==========
平均正确率: 63.75%


,股票代码,正确字段数,总字段数,正确率
0,300862,0,40,0.0
1,002967,38,40,95.0
2,300789,40,40,100.0
3,300368,0,40,0.0
4,300570,5,40,12.5
5,002350,40,40,100.0
6,300294,20,40,50.0
7,300591,37,40,92.5
8,000158,36,40,90.0
9,002949,39,40,97.5
